# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/jack/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/decl-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/

### Bidding

In [2]:
vuln_ns, vuln_ew = False, False
hand = 'AQJ986.QJ952.7.8'

auction = ["PASS", "1N"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Loaded lib dds.dll


[{'call': '2S', 'insta_score': 0.143, 'expected_score': -45, 'adjustment': 7},
 {'call': '2C', 'insta_score': 0.741, 'expected_score': -99, 'adjustment': 37}]

In [3]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


KTx.T8x.QT8xx.xx xx.Kx.KJ9xx.KJTx AQJ98x.QJ9xx.x.8 xx.Axx.Ax.AQ9xxx 0.74986 
 P-1N-2C-X-P-2D-2S-3C-P-3D-P-4C-P-P-P (11, 10.7, 11.0)  
 P-1N-2S-3C-P-3S-P-4N-P-5C-P-P-P (11, 10.7, 11.0) 
Txx.Ax.8xxx.J9xx Kxx.K8x.AKTx.Kxx AQJ98x.QJ9xx.x.8 x.Txx.QJ9x.AQTxx 0.99993 
 P-1N-2C-X-P-2D-2S-3D-3S-P-P-4D-P-P-P (10, 9.8, 9.8)  
 P-1N-2S-2N-P-3C-P-3N-P-4D-P-5C-P-P-P (9, 9.3, 9.7) 
xx.xxx.KJx.KQJxx Txxx.AKx.AQx.Axx AQJ98x.QJ9xx.x.8 K.T8.T98xxx.T9xx 0.99644 
 P-1N-2C-2D-3C-3D-P-P-P (9, 9.1, 8.5)  
 P-1N-2S-P-P-P (7, 7.9, 7.4) 
Txx.Tx.AQ8.QJTxx Kxx.Axxx.KJx.Axx AQJ98x.QJ9xx.x.8 x.K8.T9xxxx.K9xx 0.99974 
 P-1N-2C-2D-3C-P-P-3D-P-P-P (9, 9.5, 9.0)  
 P-1N-2S-2N-3S-P-P-4D-P-P-P (9, 9.5, 9.1) 
x.A8xxx.Txx.Qxxx xxx.KT.AK98.K9xx AQJ98x.QJ9xx.x.8 KTx.x.QJxxx.AJTx 0.74953 
 P-1N-2C-2D-3C-3D-P-3N-P-P-P (10, 8.3, 8.8)  
 P-1N-2S-2N-P-3N-P-P-P (10, 8.3, 8.8) 
xx.8xx.QJ8xx.xxx Kx..KT9xxx.AK9xx AQJ98x.QJ9xx.x.8 Txx.AKTxx.A.QJTx 0.74994 
 P-1N-2C-X-P-3C-P-3H-P-4C-P-4H-P-5C-P-P-P (12, 11.4, 11.9)  
 P-1N-2S-3H-P-4D-P-

West always has the same cards because it's your hand and it's known to you

In [4]:
# we got some samples above. if we want more, we can always get more
auction = ["PASS", "1N"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction, 2)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

Good quality: True
KTx.T8x.QT8xx.xx xx.Kx.KJ9xx.KJTx AQJ98x.QJ9xx.x.8 xx.Axx.Ax.AQ9xxx 0.75
Txx.Ax.8xxx.J9xx Kxx.K8x.AKTx.Kxx AQJ98x.QJ9xx.x.8 x.Txx.QJ9x.AQTxx 1.00
xx.xxx.KJx.KQJxx Txxx.AKx.AQx.Axx AQJ98x.QJ9xx.x.8 K.T8.T98xxx.T9xx 1.00
Txx.Tx.AQ8.QJTxx Kxx.Axxx.KJx.Axx AQJ98x.QJ9xx.x.8 x.K8.T9xxxx.K9xx 1.00
x.A8xxx.Txx.Qxxx xxx.KT.AK98.K9xx AQJ98x.QJ9xx.x.8 KTx.x.QJxxx.AJTx 0.75
xx.8xx.QJ8xx.xxx Kx..KT9xxx.AK9xx AQJ98x.QJ9xx.x.8 Txx.AKTxx.A.QJTx 0.75
Txx.Kx.KJ98x.QT9 Kx.A8x.xxx.KJxxx AQJ98x.QJ9xx.x.8 xx.Txx.AQTx.Axxx 0.75
xx.Kx.JT8xx.Txxx KTx.Axx.A9xx.Kxx AQJ98x.QJ9xx.x.8 xx.T8x.KQx.AQJ9x 0.75
xx.xxx.A9xx.KQ9x Kx.AT8.KQT8.Jxxx AQJ98x.QJ9xx.x.8 Txx.Kx.Jxxx.ATxx 0.75
Txx.KTxx.KJ8xx.J Kxx.8.AQxx.KTxxx AQJ98x.QJ9xx.x.8 x.Axx.T9x.AQ9xxx 0.75
